To-Do's
* Need to fix and create a no-no untouchable list of stocks since my != of GME is not working. ["GME", "BRK.B", "META"]
* Drop Berkshire, Meta, & GME
* Clean up the data and make it look better. 
* Create Secondary not_ltcm file where we can use the FRED Api to get historical risk free rates so we can do a rolling window of riskfree rates and ERP for expected returns. Maybe make a screen where its Only Jensons Alpha where Actual Return is Greater than expected. 
* Redownload Data going back to 2012 March since Wrds only goes upt to March 2022 and need more data. 

In [37]:
import pandas as pd
import numpy as np 
import scipy as sp
import statsmodels.formula.api as smf
import scipy.optimize as sco
from fredapi import Fred
from tickers_list import tickers
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
pd.set_option('display.max_rows', 10)
plt.style.use('fivethirtyeight')
%matplotlib inline

Here we are cleaning the overall stock data from factset. 
- The problem we are having is that factset's numbers are not in a floating point number format (AKA Decimal). So we had to srip out the commas in the data set to convert into a float number. 

In [38]:
stock_data = pd.read_csv("Factset_Screen_Data\stock_data.csv")
# Rename columns Symbol to Ticker
stock_data.rename(columns={'Symbol':'Ticker'}, inplace=True)
# Make Ticker the index
stock_data.set_index('Ticker', inplace=True)
# Remove commas in Recent Closing Price, Market Value, Purchase Price, Common Shares Columns
stock_data['Recent Closing Price'] = stock_data['Recent Closing Price'].str.replace(',', '').astype(float)
stock_data['Market Value'] = stock_data['Market Value'].str.replace(',', '').astype(float)
stock_data['Purchase Price'] = stock_data['Purchase Price'].str.replace(',', '').astype(float)
stock_data['Common Shares'] = stock_data['Common Shares'].str.replace(',', '').astype(float)
stock_data.head(26)

,Name,Stock Exchange,RBICS Economy,Recent Closing Price,Market Value,Purchase Price,Common Shares
Ticker,,,,,,,
A,"Agilent Technologies, Inc.",NYSE,Healthcare,130.6,38651.1,43.6,324.0
AA,Alcoa Corporation,NYSE,Non-Energy Materials,39.4,7085.5,28.1,182.9
AAL,"American Airlines Group, Inc.",NASDAQ,Industrials,13.2,8578.0,46.7,507.3
AAON,"AAON, Inc.",NASDAQ,Industrials,55.6,2950.8,33.1,52.7
AAP,"Advance Auto Parts, Inc.",NYSE,Consumer Cyclicals,169.9,10211.7,169.1,73.7
...,...,...,...,...,...,...,...
ACLS,"Axcelis Technologies, Inc.",NASDAQ,Technology,52.1,1719.4,14.6,29.5
ACM,AECOM,NYSE,Industrials,70.4,9828.5,29.7,153.9
ACN,Accenture Plc Class A,NYSE,Technology,262.2,174319.5,130.8,615.6


The problem we are having is to try to get time series data in Factset of the individual stocks since we wanted to apply modern portfolio theory to our allocation. Here are some problems we ran into. 
* Factset does not have time series data for a list of stocks. 
* Yfinance has a rate limiter which is extreamly slow. In perspective it would take over 30 mins to get all data for the ticker list each call!
* Quandl for NASDAQ is not working properly. 
* But Wharton WRDS was our best option without paying for an API key from Alpha Vantage, Morning Start, ect. 

In [39]:

df1 = pd.read_csv("Wrds_Data\wrds_data_raw.csv")
df2 = df1.pivot_table(index='MthCalDt', columns="Ticker", values="MthPrc", aggfunc='first')
remove_strings_df = df2.apply(lambda x: x.str.replace(',','')) # Brute Force way to remove strings. 
#remove_strings_df.to_csv("Wrds_Data\wrds_data_clean.csv")
remove_strings_df.head(5)

Ticker,A,AA,AAL,AAON,AAP,AAPL,AAT,AAWW,AB,ABBV,...,YUMC,YY,Z,ZBH,ZBRA,ZD,ZEN,ZION,ZTS,ZWS
MthCalDt,,,,,,,,,,,,,,,,,,,,,
20170131,48.97,36.45,44.25,33.95,164.24,121.35,42.93,52.75,23.35,61.11,...,27.48,41.08,35.38,118.33,83.67,83.81,23.93,42.19,54.94,22.09
20170228,51.3,34.59,46.36,33.65,156.61,136.99,44,56.85,23.7,61.84,...,26.59,44.29,33.94,117.08,90.71,81.42,27.23,44.9,53.31,22.17
20170331,52.87,34.4,42.3,35.35,148.26,143.66,41.84,55.45,22.85,65.16,...,27.2,46.11,33.67,122.11,91.25,83.91,28.04,42,53.37,23.08
20170428,55.05,33.73,42.62,36.65,142.14,143.65,42.83,58,22.9,65.94,...,34.12,48.97,39,119.65,94.27,90.24,28.75,40.03,56.11,24.4
20170531,60.34,32.94,48.41,36.175,133.63,152.76,39.05,48.7,22.55,66.02,...,38.41,58.34,43.52,119.21,104.34,84.62,25.98,40.07,62.28,22.8


Here we are converting the cleaned data from a string into a float. The problem that I was having is that the Data had commas in the numbers where pandas was reading it. '1,560.09'

* pt_ret is the arthematic returns of the stocks
* pt_log is the natural log returns of the stocks. (Harmonic)

In [40]:
cleaned_data = remove_strings_df.astype(float)
pt_ret = cleaned_data.pct_change(1)[1:]
pt_log = np.log(cleaned_data/cleaned_data.shift(1))[1:]

Here we are adding market data and damodarans ERP for the stocks since we want a forward looking return to continue to beat the S&P500
* The Risk Free Rate is the 10yr US Treasury Note 
* Equity Risk Premium is the recent damodarn ERP. 

In [41]:
risk_free_rate = 0.0420 # 10 year treasury rate as of 10/23/2022 at 7:09pm (From Bloomberg)
equity_risk_premium = 0.0530 # Damodaran's estimate for October 2022
arth_Sigma = pt_ret.cov() # Covariance Martrix of the arthmatic returns
log_Sigma = pt_log.cov() # Covariance Matrix of the log returns


arth_pt_var = pt_ret.var()*12 # Variance of the arthmatic returns
log_pt_var = pt_log.var()*12 # Variance of the log returns


arth_pt_dvar = pt_ret[pt_ret < 0].var()*12 # Downside Variance of the arthmatic returns
log_pt_dvar = pt_log[pt_log < 0].var()*12 # Downside Variance of the log returns



log_Sigma

Ticker,A,AA,AAL,AAON,AAP,AAPL,AAT,AAWW,AB,ABBV,...,YUMC,YY,Z,ZBH,ZBRA,ZD,ZEN,ZION,ZTS,ZWS
Ticker,,,,,,,,,,,,,,,,,,,,,
A,0.004302,0.004624,0.003335,0.002292,0.002240,0.001558,0.002057,0.003055,0.002805,0.002142,...,0.002127,0.001849,0.002147,0.002376,0.004292,0.001901,0.001888,0.002286,0.001960,0.003196
AA,0.004624,0.035389,0.012813,0.001398,0.008947,0.002441,0.011458,0.011509,0.010156,0.006601,...,0.004102,0.000853,0.009487,0.007739,0.007634,0.004690,0.005006,0.012673,0.002543,0.010545
AAL,0.003335,0.012813,0.016325,0.002670,0.004528,-0.001055,0.006681,0.003653,0.006533,0.003294,...,0.002859,0.005509,0.005668,0.004139,0.006189,0.004720,0.004432,0.008674,0.001677,0.007185
AAON,0.002292,0.001398,0.002670,0.005714,0.001630,0.002205,0.001847,0.002249,0.002037,0.000928,...,0.001273,0.001327,0.003277,0.001686,0.003447,0.001122,0.002044,0.001614,0.002051,0.001757
AAP,0.002240,0.008947,0.004528,0.001630,0.010167,0.001393,0.004376,0.004176,0.004999,0.002945,...,0.001409,0.000077,0.004841,0.003339,0.004350,0.002739,0.002914,0.004717,0.002904,0.003160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZD,0.001901,0.004690,0.004720,0.001122,0.002739,-0.003665,0.003598,0.000891,0.002858,0.001090,...,0.001916,0.000264,0.005429,0.003132,0.003497,0.006257,0.003257,0.003216,0.000972,0.004113
ZEN,0.001888,0.005006,0.004432,0.002044,0.002914,0.001534,0.003439,0.003100,0.003461,0.002252,...,0.001614,0.003695,0.006104,0.003618,0.004501,0.003257,0.008993,0.003649,0.001241,0.005496
ZION,0.002286,0.012673,0.008674,0.001614,0.004717,0.002296,0.005342,0.004257,0.005838,0.002905,...,0.002663,0.003931,0.005152,0.003892,0.005338,0.003216,0.003649,0.008934,0.001065,0.005357


Here is our regression to get the beta and alpha on each stock.

In [42]:
total_data = pd.DataFrame()
for column in pt_ret:
    model = smf.ols(f'{column} ~ VOO', data=pt_ret).fit()
    beta = model.params['VOO']
    alpha = model.params['Intercept']
    stderrA = model.bse['Intercept']
    stderrB = model.bse['VOO']
    rsquared = model.rsquared
    stats_data = pd.DataFrame({'Beta': [beta], 'Alpha': [alpha], 'StdErrA': [stderrA], 'StdErrB': [stderrB], 'R-squared':[rsquared]}, index=[column])
    total_data = pd.concat([total_data, stats_data])

total_data.index = total_data.index.rename('Ticker')
capm = ((risk_free_rate + total_data['Beta']) * equity_risk_premium)
adjCapm = ((risk_free_rate + total_data['Beta'] + total_data['Alpha']) * equity_risk_premium)
sharpe_ratio_arth = ((risk_free_rate + capm)/np.sqrt(arth_pt_var))
sharpe_ratio_log = ((risk_free_rate + capm)/np.sqrt(log_pt_var))
sortino_ratio_arth = ((risk_free_rate + capm)/np.sqrt(arth_pt_dvar))
sortino_ratio_log = ((risk_free_rate + capm)/np.sqrt(log_pt_dvar))
total_data['CAPM'] = capm
total_data['AdjCAPM'] = adjCapm
total_data['Sharpe Ratio (Arth)'] = sharpe_ratio_arth
total_data['Sharpe Ratio (Log)'] = sharpe_ratio_log
total_data['Sortino Ratio (Arth)'] = sortino_ratio_arth
total_data['Sortino Ratio (Log)'] = sortino_ratio_log

total_data = total_data.join(stock_data[['Market Value', 'Common Shares', 'Purchase Price', 'Recent Closing Price']])
total_data = total_data[(total_data['Alpha'] > 0) & (total_data['Beta'] < 3) & (total_data['CAPM'] > risk_free_rate)]
total_data.head(100)

,Beta,Alpha,StdErrA,StdErrB,R-squared,CAPM,AdjCAPM,Sharpe Ratio (Arth),Sharpe Ratio (Log),Sortino Ratio (Arth),Sortino Ratio (Log),Market Value,Common Shares,Purchase Price,Recent Closing Price
Ticker,,,,,,,,,,,,,,,
A,1.043639,0.005625,0.006146,0.132468,0.508476,0.057539,0.057837,0.434896,0.438112,0.724044,0.675495,38651.1,324.0,43.6,130.6
AA,2.261134,0.003817,0.019419,0.418547,0.327244,0.122066,0.122268,0.265421,0.251763,0.401574,0.293522,7085.5,182.9,28.1,39.4
AAPL,0.769064,0.008071,0.015621,0.336695,0.080000,0.042986,0.043414,0.199866,0.140746,0.180155,0.105674,2288636.0,21344.7,28.2,142.4
AAWW,1.161142,0.002716,0.015660,0.337528,0.164747,0.063767,0.063910,0.236417,0.222814,0.316017,0.252313,2829.0,25.0,52.2,99.9
ABBV,0.773931,0.009830,0.010051,0.216639,0.175398,0.043244,0.043765,0.294974,0.299273,0.460533,0.420269,255330.8,1592.5,62.6,144.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CHGG,0.941146,0.026201,0.018202,0.392320,0.087519,0.052107,0.053495,0.189157,0.174827,0.235200,0.168261,2750.4,91.7,7.4,21.8
CHH,1.419195,0.001364,0.006388,0.137685,0.639088,0.077443,0.077516,0.430237,0.410904,0.487140,0.408638,6800.1,56.3,56.1,121.9
CI,0.857460,0.000497,0.008980,0.193552,0.246477,0.047671,0.047698,0.331998,0.338911,0.640587,0.592715,88459.3,256.9,133.4,289.9


Now Here is our Stats Screen where 
1. Purchase Price has to be greater than the Recent Closing Price  
2. Take the 100 Largest Soritno Stocks with positive alpha only and pair them to the least correlated stock.
3. Take the 100 Largest Market Cap Stocks and pair them with the least correlated stocks in the data. 


In [43]:
# Here is the Correlation of Returns
correlation_of_arth_returns = pt_ret.corr() # Correlation Matrix of the arthmatic returns
correlation_of_log_returns = pt_log.corr() # Correlation Matrix of the log returns


# Purchase Price has to be less than recent closing price, Alpha Greater than 0 
data_screen = total_data[total_data['Purchase Price'] < total_data['Recent Closing Price']]


# Here we are getting the larget sortino ratio stocks 
largest_arth_sortino = data_screen.nlargest(100, 'Sortino Ratio (Arth)')
largest_log_sortino = data_screen.nlargest(100, 'Sortino Ratio (Log)')


largest_sortino = largest_log_sortino.head(100)

# Here we are getting the list of the larget sortino ratio stocks
tickers_list_of_largest_sortino = largest_sortino.index.tolist()


lowest_correlation = pd.DataFrame()
for ticker in tickers_list_of_largest_sortino:
    lowest_correlation = pd.concat([lowest_correlation,
     pd.DataFrame({'Ticker': [ticker], 'Paired Tickers': [correlation_of_log_returns[ticker].nsmallest(2).index[1]],
     'Correlation': [correlation_of_log_returns[ticker].nsmallest(2)[1]]})])
large_sortino_lowest_correlation = lowest_correlation
average_correlation = lowest_correlation['Correlation'].mean()


lowest_correlation.head(10)


,Ticker,Paired Tickers,Correlation
0,IVT,DVN,-0.889210
0,LIN,CTRA,-0.161139
0,MSFT,CTRA,-0.186362
0,NOW,CALM,-0.213157
0,ORCL,MKC,-0.195300
0,JBHT,VTRS,-0.118894
0,UNH,CLDX,-0.153939
0,ROP,VIRT,-0.172423
0,SPGI,CTRA,-0.129817
0,ICE,KDNY,-0.115566


Here we are taking the top 100 Market Cap Stocks in the NASDAQ & NSYE. 

In [44]:
# Top 50 Largest Market Cap Stocks
largest_market_cap = stock_data.nlargest(100, 'Market Value')
largest_market_cap = largest_market_cap[largest_market_cap['Purchase Price'] < largest_market_cap['Recent Closing Price']]
# Remove BRK.B from the list of largest_market_cap
largest_market_cap = largest_market_cap[largest_market_cap.index != 'BRK.B']

ticker_list_of_largest_market_cap = largest_market_cap.index.tolist()
largest_market_cap

,Name,Stock Exchange,RBICS Economy,Recent Closing Price,Market Value,Purchase Price,Common Shares
Ticker,,,,,,,
AAPL,Apple Inc.,NASDAQ,Technology,142.4,2288636.0,28.2,21344.7
MSFT,Microsoft Corporation,NASDAQ,Technology,237.5,1771473.0,68.9,7708.0
GOOGL,Alphabet Inc. Class A,NASDAQ,Technology,100.0,1309220.0,39.6,13825.9
AMZN,"Amazon.com, Inc.",NASDAQ,Consumer Non-Cyclicals,113.8,1159242.0,37.5,9540.0
TSLA,Tesla Inc,NASDAQ,Consumer Cyclicals,219.4,687326.7,14.2,2423.4
...,...,...,...,...,...,...,...
EOG,"EOG Resources, Inc.",NYSE,Energy,120.9,70852.8,101.1,576.7
NOW,"ServiceNow, Inc.",NYSE,Technology,348.6,70419.2,74.3,167.4
ZTS,"Zoetis, Inc. Class A",NYSE,Healthcare,149.8,70122.5,53.5,492.9


Now We are Getting the Largest Market Cap Stocks and pairing it with the least negative in our overall initial screen. 

In [45]:
lowest_correlation = pd.DataFrame()
for ticker in ticker_list_of_largest_market_cap:
    lowest_correlation = pd.concat([lowest_correlation,
     pd.DataFrame({'Ticker': [ticker], 'Paired Tickers': [correlation_of_log_returns[ticker].nsmallest(2).index[1]],
     'Correlation': [correlation_of_log_returns[ticker].nsmallest(2)[1]]})])
large_mkcap_lowest_correlation = lowest_correlation
average_correlation = lowest_correlation['Correlation'].mean()
large_mkcap_lowest_correlation.head(100)


,Ticker,Paired Tickers,Correlation
0,AAPL,CPE,-0.454709
0,MSFT,CTRA,-0.186362
0,GOOGL,CALM,-0.245629
0,AMZN,COOP,-0.295839
0,TSLA,LXU,-0.276354
...,...,...,...
0,EOG,DLR,-0.257990
0,NOW,CALM,-0.213157
0,ZTS,AVAV,-0.234531
0,SO,VIRT,-0.267323


Here we are concatinating both datasets with the Highest Sortino and Highest Makret Cap.
If there aare Stocks are in both datasets, then we are droping them so they are not repeated. 

In [46]:
# Concatenate large_sortino_lowest_correlation and large_mkcap_lowest_correlation using pd.concat and remove duplicates. 
concat_tickers = pd.concat([large_sortino_lowest_correlation, large_mkcap_lowest_correlation], ignore_index=False).drop_duplicates()
# Set index to Ticker
concat_tickers = concat_tickers.set_index('Ticker')
# Drop GME from the list of concat_tickers in the paired tickers column
cleaned_ticker_list = concat_tickers[concat_tickers['Paired Tickers'] != 'GME']
total_average_correlation = cleaned_ticker_list['Correlation'].mean()
print(f'The Average Total Correlation: {total_average_correlation} \n')
print(cleaned_ticker_list.head(10))

# Create a list of the tickers in concat_tickers from both the Ticker and Paired Tickers columns
total_ticker_list = concat_tickers.index.tolist() + concat_tickers['Paired Tickers'].tolist()

# Filter total_data to only include the tickers in total_ticker_list
filtered_total_data = total_data[total_data.index.isin(total_ticker_list)]
print(f'Total Stocks: {filtered_total_data.shape[0]} \n')
filtered_total_data.head(10)


The Average Total Correlation: -0.2192632222266268 

       Paired Tickers  Correlation
Ticker                            
IVT               DVN    -0.889210
LIN              CTRA    -0.161139
MSFT             CTRA    -0.186362
NOW              CALM    -0.213157
ORCL              MKC    -0.195300
JBHT             VTRS    -0.118894
UNH              CLDX    -0.153939
ROP              VIRT    -0.172423
SPGI             CTRA    -0.129817
ICE              KDNY    -0.115566
Total Stocks: 133 



,Beta,Alpha,StdErrA,StdErrB,R-squared,CAPM,AdjCAPM,Sharpe Ratio (Arth),Sharpe Ratio (Log),Sortino Ratio (Arth),Sortino Ratio (Log),Market Value,Common Shares,Purchase Price,Recent Closing Price
Ticker,,,,,,,,,,,,,,,
A,1.043639,0.005625,0.006146,0.132468,0.508476,0.057539,0.057837,0.434896,0.438112,0.724044,0.675495,38651.1,324.0,43.6,130.6
AAPL,0.769064,0.008071,0.015621,0.336695,0.080000,0.042986,0.043414,0.199866,0.140746,0.180155,0.105674,2288636.0,21344.7,28.2,142.4
ABBV,0.773931,0.009830,0.010051,0.216639,0.175398,0.043244,0.043765,0.294974,0.299273,0.460533,0.420269,255330.8,1592.5,62.6,144.4
ACN,1.233919,0.004880,0.004850,0.104530,0.699014,0.067624,0.067882,0.474972,0.474231,0.727168,0.672191,174319.5,615.6,130.8,262.2
ADBE,1.035593,0.012887,0.007837,0.168918,0.385159,0.057112,0.057795,0.379810,0.383291,0.666295,0.616152,136448.1,494.3,99.7,293.5
ADI,1.099440,0.002251,0.007558,0.162903,0.431547,0.060496,0.060616,0.391614,0.395973,0.644889,0.590603,71555.2,308.2,63.5,139.1
ADP,0.841766,0.004983,0.006795,0.146450,0.355098,0.046840,0.047104,0.402158,0.400606,0.531094,0.493807,96796.2,445.0,102.5,233.1
AGCO,1.278975,0.001620,0.008136,0.175360,0.469937,0.070012,0.070098,0.383909,0.388127,0.663967,0.596959,8147.6,79.5,57.9,109.2
ALL,0.813648,0.001806,0.006368,0.137246,0.369391,0.045349,0.045445,0.417228,0.419452,0.721807,0.678011,36084.5,366.0,74.1,133.5


Here were are going to do the post modern portfolio theory where we are going to create an optimized portfolio with the highest sortino ratio.

In [47]:
expectedReturn = filtered_total_data['CAPM']
# Get the downside covariance matix  less than 0 from pt_log with the ticker in filtered total data
downside_covariance = pt_log[pt_log < 0]
Sigma = downside_covariance[filtered_total_data.index].cov()*12

# Here is the bound for the minimum and maximum weights for each stock.
min_weight = 0.001
max_weight = 0.08

# Here is the number of simulations of the stocks
num_port = 100000



def portfolio_annualised_performance(weights, expectedReturns, Sigma):
    returns = np.sum(expectedReturns*weights )
    std = np.sqrt(np.dot(weights.T, np.dot(Sigma, weights)))
    return std, returns

def neg_sharpe_ratio(weights, expectedReturns, Sigma, risk_free_rate):
    p_var, p_ret = portfolio_annualised_performance(weights, expectedReturns, Sigma)
    return -(p_ret - risk_free_rate) / p_var

def max_sharpe_ratio(expectedReturns, Sigma, risk_free_rate):
    num_assets = len(expectedReturns)
    args = (expectedReturns, Sigma, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (min_weight,max_weight)
    bounds = tuple(bound for asset in range(num_assets))

    result = sco.minimize(neg_sharpe_ratio, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def portfolio_volatility(weights, expectedReturns, Sigma):
    return portfolio_annualised_performance(weights, expectedReturns, Sigma)[0]

def min_variance(expectedReturns, Sigma):
    num_assets = len(expectedReturns)
    args = (expectedReturns, Sigma)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (min_weight,max_weight)
    bounds = tuple(bound for asset in range(num_assets))

    result = sco.minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)

    return result


def display_ef_with_selected(expectedReturns, Sigma, risk_free_rate):
    max_sharpe = max_sharpe_ratio(expectedReturns, Sigma, risk_free_rate)
    max_sharpe_allocation = pd.DataFrame({"+SharpeWeight":max_sharpe.x})

    min_vol = min_variance(expectedReturns, Sigma)
    min_vol_allocation = pd.DataFrame({'-VolWeight':min_vol.x})

    
    wWeight = np.zeros((num_port,len(expectedReturns)))
    expectedReturn = np.zeros(num_port)
    expectedVolatility = np.zeros(num_port)
    sharpeRatio = np.zeros(num_port)

    markowiz_dataframe = pd.DataFrame()
    for k in range(num_port):
        # Generate random weight vector
        w = np.array(np.random.random(len(expectedReturns)))
        w = w / np.sum(w)
        wWeight[k,:] = w
        expectedReturn[k] = np.sum((expectedReturns * w))
        expectedVolatility[k] = np.sqrt(np.dot(w.T,np.dot(Sigma, w)))
        sharpeRatio[k] = expectedReturn[k]/expectedVolatility[k]

    maxIndex = sharpeRatio.argmax()
    lmx = wWeight[maxIndex,:]
    bruteAllocation = pd.DataFrame({'bruteAllocation': lmx})


    # Concatenate the max_sharpe_allocation, min_vol_allocation, and bruteAllocation DataFrames
    markowiz_dataframe = pd.concat([max_sharpe_allocation, min_vol_allocation, bruteAllocation], axis=1)

    # Concatenate the filtered_total_data and markowiz_dataframe DataFrames on the filtered_total_data index
    markowiz_dataframe = pd.concat([filtered_total_data.reset_index(), markowiz_dataframe],axis=1)
    markowiz_dataframe = markowiz_dataframe.set_index('Ticker')
    

    return markowiz_dataframe
    

efficent_frontier_data = display_ef_with_selected(expectedReturn, Sigma, risk_free_rate)



# Check to See if the Data sums to 1
print(f"To Check if Summed to One: {efficent_frontier_data[['+SharpeWeight']].sum().to_string()}")
print(f"To Check if Summed to One: {efficent_frontier_data[['-VolWeight']].sum().to_string()}")
print(f"To Check if Summed to One: {efficent_frontier_data[['bruteAllocation']].sum().to_string()} \n")


portfolio_expected_return_Scipy = np.sum(efficent_frontier_data['CAPM']*efficent_frontier_data['+SharpeWeight'])
portfolio_expected_return_brute = np.sum(efficent_frontier_data['CAPM']*efficent_frontier_data['bruteAllocation'])
portfolio_expected_return_MinVol = np.sum(efficent_frontier_data['CAPM']*efficent_frontier_data['-VolWeight'])

portfolio_volatility_Scipy = np.sqrt(np.dot(efficent_frontier_data['+SharpeWeight'].T,np.dot(Sigma, efficent_frontier_data['+SharpeWeight'])))
portfolio_volatility_brute = np.sqrt(np.dot(efficent_frontier_data['bruteAllocation'].T,np.dot(Sigma, efficent_frontier_data['bruteAllocation'])))
portfolio_volatility_MinVol = np.sqrt(np.dot(efficent_frontier_data['-VolWeight'].T,np.dot(Sigma, efficent_frontier_data['-VolWeight'])))

portfolio_sortino_Scipy = portfolio_expected_return_Scipy/portfolio_volatility_Scipy
print(portfolio_volatility_Scipy)


print(f"Portfolio Expected Return Scipy: {portfolio_expected_return_Scipy}")
# print(f"Portfolio Sortino Ratio Scipy: {portfolio_expected_return_Scipy/portfolio_volatility_Scipy} /n")
print(f"Portfolio Expected Return Brute: {portfolio_expected_return_brute}")

print(f"Portfolio Expected Return MinVol: {portfolio_expected_return_MinVol}")


# Sort by brite force allocation for the largest weights
sorted_brute_allocation = efficent_frontier_data.sort_values(by=['bruteAllocation'], ascending=False)
sorted_brute_allocation


To Check if Summed to One: +SharpeWeight    1.0
To Check if Summed to One: -VolWeight    1.0
To Check if Summed to One: bruteAllocation    1.0 

nan
Portfolio Expected Return Scipy: 0.05974469955599921
Portfolio Expected Return Brute: 0.05996943753957294
Portfolio Expected Return MinVol: 0.05974469955599921


,Beta,Alpha,StdErrA,StdErrB,R-squared,CAPM,AdjCAPM,Sharpe Ratio (Arth),Sharpe Ratio (Log),Sortino Ratio (Arth),Sortino Ratio (Log),Market Value,Common Shares,Purchase Price,Recent Closing Price,+SharpeWeight,-VolWeight,bruteAllocation
Ticker,,,,,,,,,,,,,,,,,,
UNP,1.164068,0.003433,0.005884,0.126823,0.584051,0.063922,0.064104,0.444671,0.454010,0.737966,0.695980,124021.5,815.8,103.7,198.6,0.007519,0.007519,0.014902
REXR,0.777964,0.011778,0.006794,0.146434,0.319923,0.043458,0.044082,0.397305,0.404797,0.674707,0.637095,9061.4,66.5,23.2,53.0,0.007519,0.007519,0.014824
PKI,1.081734,0.009208,0.008218,0.177125,0.383335,0.059558,0.060046,0.371698,0.378516,0.626414,0.580500,15983.7,112.0,52.2,126.6,0.007519,0.007519,0.014689
MCO,1.212383,0.006724,0.005169,0.111401,0.663753,0.066482,0.066839,0.466154,0.466322,0.759896,0.708790,44601.5,190.7,94.3,243.1,0.007519,0.007519,0.014554
KURA,1.392703,0.015518,0.026184,0.564360,0.092145,0.076039,0.076862,0.164517,0.191044,0.413680,0.343794,1028.8,19.3,5.9,15.4,0.007519,0.007519,0.014498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ABBV,0.773931,0.009830,0.010051,0.216639,0.175398,0.043244,0.043765,0.294974,0.299273,0.460533,0.420269,255330.8,1592.5,62.6,144.4,0.007519,0.007519,0.000302
LHX,0.782468,0.007375,0.008398,0.181009,0.237482,0.043697,0.044088,0.341287,0.348072,0.650335,0.605432,42778.9,124.6,82.6,223.6,0.007519,0.007519,0.000186
ACN,1.233919,0.004880,0.004850,0.104530,0.699014,0.067624,0.067882,0.474972,0.474231,0.727168,0.672191,174319.5,615.6,130.8,262.2,0.007519,0.007519,0.000091
